
# Lesson 9: Recommender System - Recommend Books

Recommendation Systems usually rely on larger datasets and specifically need to be organized in a particular fashion.

## The Data

You will use data from http://www2.informatik.uni-freiburg.de/~cziegler/BX/
____

Objective is to recommend books to a user based on purchase history and behavior of other users

## Getting Started

Import some libraries you will need:

In [ ]:
import numpy as np
import pandas as pd

Read the data using pandas in DataFrame df

In [ ]:
df_user = pd.read_csv('BX-Users.csv',encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Take a quick look at the user data.

In [ ]:
df_user.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Clean up NaN values

Read the books Data and explore

In [ ]:
#column_names = ['isbn', 'book_title']
df_books = pd.read_csv('BX-Books.csv', encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
for book_name in df_books['book_title'].unique():
  print('book_name: ', book_name)

Выходные данные были обрезаны до нескольких последних строк (5000).
book_name:  Spinoza of Market St
book_name:  Balancing Act
book_name:  Great Baseball Feats, Facts &amp; Firsts (Great Baseball Feats, Facts &amp; Firsts)
book_name:  Before Freedom: 48 Oral Histories of Former North and South Carolina Slaves
book_name:  Montgomery's Children: A Novel (Plume Fiction)
book_name:  Lay Bare the Heart: An Autobiography of the Civil Rights Movement
book_name:  Buffalo gals and other animal presences
book_name:  Country Cured: Reflections from the Heart
book_name:  The Sea of Light (Plume Books)
book_name:  Women in Evidence
book_name:  Benchley Lost and Found: Thirty-Nine Prodigal Pieces
book_name:  Snow White and Other Fairy Tales (Dover Children's Thrift Classics)
book_name:  A Little Princess (Dover Children's Thrift Classics)
book_name:  Mrs. Peter Rabbit (Children's Thrift Classics)
book_name:  Art of the Andes: From Chavin to Inca (World of Art)
book_name:  The Thames and Hudson Manua

Now, read the data where ratings are given by users. You will read only first 10K rows otherwise, Out Of Memory  error can occur.

In [ ]:
df = pd.read_csv('BX-Book-Ratings.csv',encoding='latin-1',nrows=10000)

In [ ]:
df.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [ ]:
df.describe()

,user_id,rating
count,10000.000000,10000.000000
mean,265844.379600,1.974700
std,56937.189618,3.424884
min,2.000000,0.000000
25%,277478.000000,0.000000
50%,278418.000000,0.000000
75%,278418.000000,4.000000
max,278854.000000,10.000000


Merge the dataframes. For all practical purposes, User Master Data is not required. So, ignore dataframe df_user

In [ ]:
df = pd.merge(df,df_books,on='isbn')
df.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276726,155061224,5,Rites of Passage,Judith Rae,2001,Heinle
2,276727,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
3,278418,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
4,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press


Now, let's take a quick look at the number of unique users and books.

In [ ]:
n_users = df.user_id.nunique()
n_books = df.isbn.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Books: '+str(n_books))

Num. of Users: 828
Num of Books: 8051


Convert ISBN to numeric numbers in order

In [ ]:
isbn_list = df.isbn.unique()
print(" Length of isbn List:", len(isbn_list))
def get_isbn_numeric_id(isbn):
    #print ("  isbn is:" , isbn)
    itemindex = np.where(isbn_list==isbn)
    return itemindex[0][0]

 Length of isbn List: 8051


Do the same for user_id , convert it into numeric and in order

In [ ]:
userid_list = df.user_id.unique()
print(" Length of user_id List:", len(userid_list))
def get_user_id_numeric_id(user_id):
    #print ("  isbn is:" , isbn)
    itemindex = np.where(userid_list==user_id)
    return itemindex[0][0]

 Length of user_id List: 828


Converting both user_id and isbn to ordered list i.e. from 0...n-1

In [ ]:
df['user_id_order'] = df['user_id'].apply(get_user_id_numeric_id)

In [ ]:
df['isbn_id'] = df['isbn'].apply(get_isbn_numeric_id)
df.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,user_id_order,isbn_id
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,0,0
1,276726,155061224,5,Rites of Passage,Judith Rae,2001,Heinle,1,1
2,276727,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,2,2
3,278418,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,3,2
4,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,4,3


Re-index columns to build matrix later on

In [ ]:
new_col_order = ['user_id_order', 'isbn_id', 'rating', 'book_title', 'book_author','year_of_publication','publisher','isbn','user_id']
df = df.reindex(columns= new_col_order)
df.head()

,user_id_order,isbn_id,rating,book_title,book_author,year_of_publication,publisher,isbn,user_id
0,0,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,034545104X,276725
1,1,1,5,Rites of Passage,Judith Rae,2001,Heinle,155061224,276726
2,2,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,446520802,276727
3,3,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,446520802,278418
4,4,3,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,052165615X,276729


## Train Test Split

Recommendation Systems  are difficult to evaluate, but you will still learn how to evaluate them. In order to do this, you'll split your data into two sets. However, you won't do your classic X_train,X_test,y_train,y_test split. Instead, you can actually just segement the data into two sets of data:

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.30)

## Approach: You Will Use Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**.

A *user-item filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked.

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items as input and outputs other items as recommendations.

* *Item-Item Collaborative Filtering*: “Users who liked this item also liked …”
* *User-Item Collaborative Filtering*: “Users who are similar to you also liked …”

In both cases, you create a user-book matrix which is built from the entire dataset.

Since you have split the data into testing and training, you will need to create two ``[828 x 8051]`` matrices (all users by all books). This is going to be  a very large matrix

The training matrix contains 70% of the ratings and the testing matrix contains 30% of the ratings.  

In [ ]:
#Create two user-book matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_books))
for _l, user_id_order_l, isbn_id_l, rating_l, *_l in train_data.itertuples():
    train_data_matrix[user_id_order_l-1, isbn_id_l-1] = rating_l

test_data_matrix = np.zeros((n_users, n_books))
for _l, user_id_order_l, isbn_id_l, rating_l, *_l in test_data.itertuples():
    test_data_matrix[user_id_order_l-1, isbn_id_l-1] = rating_l

You can use the [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) function from sklearn to calculate the cosine similarity. Note, the output will range from 0 to 1 since the ratings are all positive.

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [ ]:
user_similarity

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

Next step is to make predictions

In [ ]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Evaluation
There are many evaluation metrics, but one of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error (RMSE)*.

Since, you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with: `prediction[ground_truth.nonzero()]`.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [ ]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 7.650882345491601
Item-based CF RMSE: 7.650323511867364


### Both the approach yield almost same result

# End